In [ ]:
%%time

import pyspark
import dxpy
import hail as hl

my_database = dxpy.find_one_data_object(
    name="my_database", 
    project=dxpy.find_one_project()["id"]
)["id"]
database_dir = f'dnax://{my_database}'
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)
hl.init(sc=sc, tmp_dir=f'{database_dir}/tmp/')


In [ ]:
%%time

chrom = 21
block = '*'

vcf_dir='file:///mnt/project/Bulk/Exome sequences_Alternative exome processing/Exome variant call files (gnomAD) (VCFs)'
vcf_path=f'{vcf_dir}/ukb24068_c{chrom}_b{block}_v1.vcf.gz'

# SPECIFIC TO CHR21
assert len(hl.hadoop_ls(vcf_path))==400

raw = hl.import_vcf(
    vcf_path, 
    force_bgz=True,
    reference_genome='GRCh38'
)

In [ ]:
def site_filter(mt):
    MIN_DP = 10
    MIN_GQ = 20

    pass_dp = mt.DP>=MIN_DP
    pass_gq = mt.GQ>=MIN_GQ

    pass_ab_het = mt.GT.is_het() & (mt.AD[1]/mt.DP>0.2)
    pass_ab = ~mt.GT.is_het() | pass_ab_het
    mt = mt.filter_entries(pass_dp & pass_gq & pass_ab)

    return mt

def get_filter_contains_rf_expr(mt):
    return mt.filters.contains('RF')

def get_fail_interval_qc_expr(mt):
    return mt.info.fail_interval_qc

def get_lcr_expr(mt):
    return mt.info.lcr

def get_segdup_expr(mt):
    return mt.info.segdup

def variant_filter_v1_0(mt):
    return mt.filter_rows(
        get_filter_contains_rf_expr(mt),
        keep=False
    )

def variant_filter_v1_1(mt):
    return mt.filter_rows(
        get_filter_contains_rf_expr(mt)
        | get_fail_interval_qc_expr(mt)
        | get_lcr_expr(mt)
        | get_segdup_expr(mt),
        keep=False
    )

def variant_filter_v1_2(mt):
    return mt.filter_rows(
        get_fail_interval_qc_expr(mt)
        | get_lcr_expr(mt)
        | get_segdup_expr(mt),
        keep=False
    )

def export_table(ht, fname, out_folder):
    ht.naive_coalesce(1).export(f'file:///opt/notebooks/{fname}')

    dxpy.upload_local_file(
        filename=f'/opt/notebooks/{fname}',
        name=fname,
        folder=out_folder,
        parents=True
    )

In [ ]:
%%time

filtered = variant_filter_v1_2(raw)
filtered = site_filter(filtered)
filtered = hl.sample_qc(filtered)

cols = filtered.cols()
cols = cols.transmute(**cols.sample_qc.flatten())

export_table(
    ht=cols,
    fname=f'sample_qc_c{chrom}.v1.2.papermill.tsv.gz',
    out_folder='/data/01_get_sample_stats'
)